In [1]:
# import libraries

import os
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import mygene
import sklearn.preprocessing as sk
import seaborn as sns
from sklearn import metrics
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from torch.utils.data.sampler import WeightedRandomSampler
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score, precision_recall_curve
import random
from random import randint
from sklearn.model_selection import StratifiedKFold
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [2]:
# define directories

cell_line_dir = "/Volumes/Expansion/Thesis Work/Supplementary Files/GDSC/"
models_dir = "/Volumes/Expansion/Thesis Work/Results/Transfer Learning/Drugs with same pathways/TCGA_DNA_REPi/Expression_Mutation_CNA/Models/"
DEGs_dir = "/Volumes/Expansion/Thesis Work/Results/GDSC_DEGs_inhibitors/DNA_REPi/"
dataset_dir = "/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/"
save_results_to = "/Volumes/Expansion/Thesis Work/Results/Transfer Learning/Drugs with same pathways/TCGA_DNA_REPi/Expression_Mutation_CNA/Predictions/"

In [3]:
# set random seeds

torch.manual_seed(42)
random.seed(42)

In [4]:
# change directory to read GDSC cell line details

os.chdir(cell_line_dir)
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Supplementary Files/GDSC


In [5]:
# read GDSC cell line details table

GDSC_cell_line_details = pd.read_excel("GDSC_Cell_Lines_Details.xlsx", keep_default_na = False)
GDSC_cell_line_details.set_index("COSMIC identifier", inplace = True)
GDSC_cell_line_details.tail()

,Sample Name,Whole Exome Sequencing (WES),Copy Number Alterations (CNA),Gene Expression,Methylation,Drug\nResponse,GDSC\nTissue descriptor 1,GDSC\nTissue\ndescriptor 2,Cancer Type\n(matching TCGA label),Microsatellite \ninstability Status (MSI),Screen Medium,Growth Properties
COSMIC identifier,,,,,,,,,,,,
1298218,KP-2,Y,N,Y,N,Y,pancreas,pancreas,PAAD,,D/F12,Adherent
1330932,KO52,Y,Y,N,N,N,leukemia,acute_myeloid_leukaemia,LAML,MSS/MSI-L,D/F12,NA
1331030,SC-1,Y,Y,N,N,N,lymphoma,B_cell_lymphoma,DLBC,MSS/MSI-L,R,NA
1503373,U-CH2,Y,Y,N,N,N,bone,bone_other,,MSS/MSI-L,D/F12,Adherent
,TOTAL:,1001,996,968,957,990,,,,,,


In [6]:
# change directory to read GDSC expression dataset (DNA_REPi)

os.chdir(dataset_dir + "/exprs/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/exprs


In [7]:
# read GDSC expression dataset (DNA_REPi)

GDSCE = pd.read_csv("GDSC_exprs.z.DNA_REPi.tsv", 
                    sep = "\t", index_col=0, decimal = ",")
GDSCE = pd.DataFrame.transpose(GDSCE)
GDSCE.head(3)

ENTREZID,1,2,9,10,12,13,14,15,16,18,...,107984199,107984208,107984325,107984411,107984648,107984889,107984989,107986313,107986782,107986810
683665,-0.4711563767951986,-0.1857522957766964,0.9702409548662462,-0.4081281805960821,-0.4161511169999803,-0.4384159723467665,-1.154692246005183,0.5262132484107975,-0.7950924576033422,-0.6337442716864214,...,1.283534690590172,-0.3367265464586691,-0.0039762988723447,0.7378729872604209,0.9757614264575688,0.1494417647095801,2.062294063911948,-0.8140636761719959,0.2870333828833719,0.9524265085537148
684055,1.3565140948247052,-0.277542411913307,0.0887087882196536,-0.2754829982302413,-0.5153856346587746,-0.6248712076433911,0.1243408990866293,-1.2699667684674136,2.0788360886382717,2.5197991997459197,...,-0.5571028630154293,-1.724090733574364,-0.3749080365068901,-0.0016055431935905,-0.3788535663547365,-1.3502092798315848,0.6949790577550328,0.4333032198982747,-0.1280986270308099,-1.629249958712223
684057,0.6510004960254864,1.6578876382433665,-0.4977106390881842,-0.1212172092933644,-0.398455281697677,-0.2804901106672752,0.6455898116420014,0.215833091774836,-0.1723595788875009,-0.2170870770552223,...,-0.024863133395904,0.4073439017665206,-0.4440356638831038,0.8630347078663075,1.4486171428489227,-1.4777214830771732,0.5220400480464272,0.6441211958947753,0.9057518606131394,0.3018783753479829


In [8]:
# change directory to read GDSC mutation dataset (DNA_REPi)

os.chdir(dataset_dir + "/mutations/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/mutations


In [9]:
# read GDSC mutation dataset (DNA_REPi)

GDSCM = pd.read_csv("GDSC_mutations.DNA_REPi.tsv",
                    sep="\t", index_col=0, decimal=".")
GDSCM.drop_duplicates(keep='last')
GDSCM = pd.DataFrame.transpose(GDSCM)
GDSCM = GDSCM.loc[:, ~GDSCM.columns.duplicated()]

GDSCM.head()

,143872,728577,2,2050,10243,79365,256006,4609,118788,113675,...,26517,28972,53917,283507,514,339210,64981,7335,5739,83442
683665,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
684055,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
684057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
684059,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
684062,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# change directory to read GDSC CNA dataset (DNA_REPi)

os.chdir(dataset_dir + "/CNA/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/CNA


In [11]:
# read GDSC CNA dataset (DNA_REPi)

GDSCC = pd.read_csv("GDSC_CNA.DNA_REPi.tsv",
                    sep="\t", index_col=0, decimal=".")
GDSCC.drop_duplicates(keep='last')
GDSCC = pd.DataFrame.transpose(GDSCC)
GDSCC = GDSCC.loc[:, ~GDSCC.columns.duplicated()]

In [12]:
# change directory to read GDSC response dataset (DNA_REPi)

os.chdir(dataset_dir + "/response/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/response


In [13]:
# read GDSC response dataset (DNA_REPi)

GDSCR = pd.read_csv("GDSC_response.DNA_REPi.tsv",
                    sep="\t",
                    index_col=0,
                    decimal=",")
GDSCR.dropna(inplace=True)
GDSCR.rename(mapper=str, axis='index', inplace=True)
d = {"R": 0, "S": 1}
GDSCR["response"] = GDSCR.loc[:, "response"].apply(lambda x: d[x])

GDSCR.head()

,response,logIC50,drug,exprs,CNA,mutations
sample_name,,,,,,
683665,0,5.12075503641523,Pyrimethamine,1,1,1
683665,0,-3.03711635474651,Doxorubicin,1,1,1
683665,0,-0.712119156515401,Etoposide,1,1,1
683665,0,-4.40897202584936,Gemcitabine,1,1,1
683665,0,-1.59752443691324,Mitomycin C,1,1,1


In [14]:
# print count of cell lines for each drug (DNA_REPi)

GDSCR["drug"].value_counts()

drug
Etoposide         775
SN-38             771
Mitomycin C       767
Doxorubicin       766
5-Fluorouracil    764
Gemcitabine       761
Bleomycin         759
Temozolomide      752
Cisplatin         738
Methotrexate      738
Camptothecin      737
Cytarabine        735
Pyrimethamine     372
Name: count, dtype: int64

In [15]:
# change directory to read TCGA expression dataset homogenized with GDSC expression dataset (Cisplatin)

os.chdir(dataset_dir + "exprs_homogenized/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/exprs_homogenized


In [16]:
# read TCGA expression dataset homogenized with GDSC expression dataset (Cisplatin)

TCGAEcisplatin = pd.read_csv("TCGA_exprs.Cisplatin.eb_with.GDSC_exprs.Cisplatin.tsv",
                   sep = "\t", index_col=0, decimal = ",")
TCGAEcisplatin = pd.DataFrame.transpose(TCGAEcisplatin)
TCGAEcisplatin.head(3)

ENTREZID,1,2,9,10,12,13,14,15,16,18,...,100188893,100190940,100190949,100191040,100192386,100268168,100271715,100287718,100288778,100289635
TCGA-3B-A9HT-01A,4.81366326511911,6.90588058929056,7.28792978013405,2.82740736787,4.160259560633,2.48501823090881,8.49326947598111,2.7294837261655,9.2940474880707,3.7263445184204,...,9.48408428341014,2.84248002627717,2.79923598179059,3.14372627923905,2.81381378171936,2.67246444782135,2.40864068637175,2.79909224555968,4.26862543327281,4.10488574519165
TCGA-B7-A5TN-01A,3.59835913173055,7.05758623769129,6.38284534468598,3.69159217764028,6.02278673577733,5.13381978626662,7.83837956597428,2.56971659684603,7.60783718355108,4.93038074450586,...,9.11203143915675,2.84248002627717,2.87676029314036,4.529732927748,2.91852230508839,2.58945785388122,2.4118120370575,2.79909224555968,3.85489562164745,5.19393132629067
TCGA-BA-A4IF-01A,3.6387513624083,3.50568983094232,7.00534226807151,2.82740736787,3.6684842125734,3.83997981618545,8.79244234307359,2.71465281463663,9.05071416661714,6.0607542432723,...,9.35461690949584,2.84248002627717,2.79923598179059,3.38499009274846,3.26675151239393,2.5682276993458,2.99841365650393,2.79909224555968,4.02262450424724,4.42294030595364


In [17]:
# change directory to read TCGA mutation dataset (Cisplatin)

os.chdir(dataset_dir + "mutations")
os.getcwd()

'/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/mutations'

In [18]:
# read TCGA mutation dataset (Cisplatin)

TCGAMcisplatin = pd.read_csv("TCGA_mutations.Cisplatin.tsv",
                        sep="\t", index_col=0, decimal=",")
TCGAMcisplatin.drop_duplicates(keep='last')
TCGAMcisplatin = pd.DataFrame.transpose(TCGAMcisplatin)
TCGAMcisplatin = TCGAMcisplatin.loc[:, ~TCGAMcisplatin.columns.duplicated()]

TCGAMcisplatin.head(3)

,11266,118788,8714,10763,387601,54810,10799,48,51251,56884,...,100529262,4831,51337,100526760,10638,282969,101927016,5912,57819,8649
TCGA-3B-A9HT-01A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-B7-A5TN-01A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-BA-A4IF-01A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
# change directory to read TCGA CNA dataset (Cisplatin)

os.chdir(dataset_dir + "CNA")
os.getcwd()

'/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/CNA'

In [20]:
# read TCGA CNA dataset (Cisplatin)

TCGACcisplatin = pd.read_csv("TCGA_CNA.Cisplatin.tsv",
                        sep="\t", index_col=0, decimal=",")
TCGACcisplatin.drop_duplicates(keep='last')
TCGACcisplatin = pd.DataFrame.transpose(TCGACcisplatin)
TCGACcisplatin = TCGACcisplatin.loc[:, ~TCGACcisplatin.columns.duplicated()]

TCGACcisplatin.head(3)

,26,27,58,88,95,103,134,137,142,159,...,100422827,100423000,100507338,100507384,100507401,100507417,100507606,100616222,100616368,100652976
TCGA-3B-A9HT-01A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-B7-A5TN-01A,-0.4166,0.0,0.0,0.0,-0.2693,0.3406,0.0,0.0,0.0,0.0,...,0.3357,0.3357,0.3357,0.3357,0.3357,0.3357,0.3357,0.3357,0.3357,0.3357
TCGA-BA-A4IF-01A,0.0,0.0,0.0,0.0,-0.3029,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
# variance threshold for GDSC Expression dataset (DNA_REPi)

selector = VarianceThreshold(0.05)
selector.fit_transform(GDSCE)
GDSCE = GDSCE[GDSCE.columns[selector.get_support(indices=True)]]

In [22]:
# fill NA values and binarize GDSC mutation and CNA datasets (DNA_REPi)

GDSCM = GDSCM.fillna(0)
GDSCM[GDSCM != 0.0] = 1
GDSCC = GDSCC.fillna(0)
GDSCC[GDSCC != 0.0] = 1

In [23]:
# select shared genes between GDSC and TCGA expression, mutation, and CNA datasets 

ls = GDSCE.columns.intersection(GDSCM.columns)
ls = ls.intersection(GDSCC.columns)
ls = ls.intersection(TCGAEcisplatin.columns)
ls = ls.intersection(TCGAMcisplatin.columns)
ls = ls.intersection(TCGACcisplatin.columns)

ls = pd.unique(ls)

print(f"GDSC and TCGA# of common genes: {ls.shape[0]}")

GDSC and TCGA# of common genes: 15850


In [24]:
# select shared samples between GDSC expression, mutation, CNA, and response datasets (DNA_REPi)

ls2 = GDSCE.index.intersection(GDSCM.index)
ls2 = ls2.intersection(GDSCC.index)
ls2 = ls2.intersection(GDSCR.index)

print(f"GDSC # of common cell lines: {ls2.shape[0]}")

GDSC # of common cell lines: 801


In [25]:
# subset shared genes and samples in GDSC expression, mutation, CNA, and response datasets

GDSCE = GDSCE.loc[ls2, ls]
GDSCM = GDSCM.loc[ls2, ls]
GDSCC = GDSCC.loc[ls2, ls]
GDSCR = GDSCR.loc[ls2, :]

print(f"GDSC # of common samples and genes (Expression): {GDSCE.shape}")
print(f"GDSC # of common samples and genes (Mutation): {GDSCM.shape}")
print(f"GDSC # of common samples and genes (CNA): {GDSCC.shape}")
print(f"GDSC # of samples (Response): {GDSCR.shape[0]}")

GDSC # of common samples and genes (Expression): (801, 15850)
GDSC # of common samples and genes (Mutation): (801, 15850)
GDSC # of common samples and genes (CNA): (801, 15850)
GDSC # of samples (Response): 9435


In [26]:
# select shared samples between TCGA expression, mutation, and CNA datasets (Cisplatin) 

ls3 = TCGAEcisplatin.index.intersection(TCGAMcisplatin.index)
ls3 = ls3.intersection(TCGACcisplatin.index)

print(f"TCGA # of common samples (Cisplatin): {ls3.shape[0]}")

TCGA # of common samples (Cisplatin): 64


In [27]:
# select shared genes and samples between TCGA expression, mutation, and CNA datasets (Cisplatin)

TCGAEcisplatin = TCGAEcisplatin.loc[ls3, ls]
TCGAMcisplatin = TCGAMcisplatin.loc[ls3, ls]
TCGACcisplatin = TCGACcisplatin.loc[ls3, ls]

print(f"TCGA # of common samples and genes for Expression (Cisplatin): {TCGAEcisplatin.shape}")
print(f"TCGA # of common samples and genes for Mutation (Cisplatin): {TCGAMcisplatin.shape}")
print(f"TCGA # of common samples and genes for CNA (Cisplatin): {TCGACcisplatin.shape}\n")

TCGA # of common samples and genes for Expression (Cisplatin): (64, 15850)
TCGA # of common samples and genes for Mutation (Cisplatin): (64, 15850)
TCGA # of common samples and genes for CNA (Cisplatin): (64, 15850)



In [28]:
# change directory to read DEGs (DNA_REPi)

os.chdir(DEGs_dir)
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/GDSC_DEGs_inhibitors/DNA_REPi


In [29]:
# read DEGs (DNA_REPi)

DEGs_filtered_data = pd.read_excel("DNA_REPi_Differentially_Expressed_Genes (EnsemblID).xlsx",
                                    sheet_name = "Common DEGs")
print(f"There are {DEGs_filtered_data.shape[0]} DEGs (p-adjusted < 0.05) among DNA replication inhibitors\n")

DEGs_filtered_data.head()

There are 858 DEGs (p-adjusted < 0.05) among DNA replication inhibitors



,Gene Symbol,Frequency
0,SLFN11,9
1,NCKAP1L,8
2,SDC4,7
3,KRT18,7
4,S100A14,7


In [30]:
# get gene symbol from mygene database

mg = mygene.MyGeneInfo()
DEGs_entrez_id = mg.querymany(DEGs_filtered_data["Gene Symbol"], 
                              species="human", 
                              scopes = "symbol",
                              field = "entrezgene",
                              as_dataframe=True)["entrezgene"]
DEGs_entrez_id = pd.unique(DEGs_entrez_id.dropna())
DEGs_entrez_id.shape[0]

1 input query terms found dup hits:	[('CLECL1P', 2)]


858

In [31]:
# assign GDSC data to new variables

exprs_z = GDSCE
mut = GDSCM
cna = GDSCC
responses = GDSCR

In [32]:
# GDSC response dataset

responses.head(3)

,response,logIC50,drug,exprs,CNA,mutations
683665,0,5.12075503641523,Pyrimethamine,1,1,1
683665,0,-3.03711635474651,Doxorubicin,1,1,1
683665,0,-0.712119156515401,Etoposide,1,1,1


In [33]:
# list drug names in GDSC response dataset

drugs = set(responses["drug"].values)
drugs

{'5-Fluorouracil',
 'Bleomycin',
 'Camptothecin',
 'Cisplatin',
 'Cytarabine',
 'Doxorubicin',
 'Etoposide',
 'Gemcitabine',
 'Methotrexate',
 'Mitomycin C',
 'Pyrimethamine',
 'SN-38',
 'Temozolomide'}

In [34]:
# convert Entrez gene IDs to integer

responses.index = responses.index.astype(int)
responses.index

Index([ 683665,  683665,  683665,  683665,  683665,  683665,  683665,  683665,
        683665,  683665,
       ...
       1659818, 1659818, 1659818, 1659818, 1659819, 1659819, 1659819, 1659819,
       1659819,  687800],
      dtype='int64', length=9435)

In [35]:
# merge GDSC cell line details with response dataset

dat = pd.merge(GDSC_cell_line_details,
         responses,
         left_index = True, 
         right_index=True,
         how = "inner")

dat["GDSC\nTissue descriptor 1"].value_counts()
#dat.head()

GDSC\nTissue descriptor 1
lung_NSCLC           1037
urogenital_system     926
leukemia              829
lymphoma              769
aero_dig_tract        727
nervous_system        607
lung_SCLC             569
skin                  560
breast                524
digestive_system      466
large_intestine       447
bone                  421
neuroblastoma         332
pancreas              294
kidney                227
soft_tissue           191
myeloma               182
lung                  174
thyroid               153
Name: count, dtype: int64

In [36]:
# get tissue descriptors

dat["GDSC\nTissue\ndescriptor 2"].value_counts().head()

GDSC\nTissue\ndescriptor 2
lung_NSCLC_adenocarcinoma    639
lung_small_cell_carcinoma    569
glioma                       558
breast                       524
melanoma                     522
Name: count, dtype: int64

In [37]:
# convert Entrez gene IDs to string

responses.index = responses.index.astype(str)
responses.index

Index(['683665', '683665', '683665', '683665', '683665', '683665', '683665',
       '683665', '683665', '683665',
       ...
       '1659818', '1659818', '1659818', '1659818', '1659819', '1659819',
       '1659819', '1659819', '1659819', '687800'],
      dtype='object', length=9435)

In [38]:
# filter GDSC expression, mutation, and CNA datasets (DNA_REPi) as to drugs
# subset selected DEGs

expression_zscores = []
CNA = []
mutations = []
for drug in drugs:
    samples = responses.loc[responses["drug"] == drug, :].index.values
    e_z = exprs_z.loc[samples, :]
    c = cna.loc[samples, :]
    m = mut.loc[samples, :]

    expression_zscores.append(e_z)
    CNA.append(c)
    mutations.append(m)

GDSCEv2 = pd.concat(expression_zscores, axis=0)
GDSCMv2 = pd.concat(mutations, axis=0)
GDSCCv2 = pd.concat(CNA, axis=0)
GDSCRv2 = responses

ls4 = list(set(GDSCE.columns).intersection(set(DEGs_entrez_id.astype(int))))
ls5 = GDSCEv2.index.intersection(GDSCMv2.index)
ls5 = ls5.intersection(GDSCCv2.index)

GDSCEv2 = GDSCEv2.loc[ls5, ls4]
GDSCMv2 = GDSCMv2.loc[ls5, ls4]
GDSCCv2 = GDSCCv2.loc[ls5, ls4]
GDSCRv2 = GDSCRv2.loc[ls5, :]

TCGAEcisplatin = TCGAEcisplatin.loc[:,ls4]
TCGAMcisplatin = TCGAMcisplatin.loc[:,ls4]
TCGACcisplatin = TCGACcisplatin.loc[:,ls4]

responses.index = responses.index.values + "_" + responses["drug"].values

print(f"GDSC # of common samples and genes (Expression): {GDSCEv2.shape}")
print(f"GDSC # of common samples and genes (Mutation): {GDSCMv2.shape}")
print(f"GDSC # of common samples and genes (CNA): {GDSCCv2.shape}")
print(f"GDSC # of common samples (Response): {GDSCRv2.shape[0]}\n")

print(f"TCGA # of common samples and genes for Cisplatin (Expression): {TCGAEcisplatin.shape}")
print(f"TCGA # of common samples and genes for Cisplatin (Mutation): {TCGAMcisplatin.shape}")
print(f"TCGA # of common samples and genes for Cisplatin (CNA): {TCGACcisplatin.shape}\n")

GDSC # of common samples and genes (Expression): (9435, 758)
GDSC # of common samples and genes (Mutation): (9435, 758)
GDSC # of common samples and genes (CNA): (9435, 758)
GDSC # of common samples (Response): 9435

TCGA # of common samples and genes for Cisplatin (Expression): (64, 758)
TCGA # of common samples and genes for Cisplatin (Mutation): (64, 758)
TCGA # of common samples and genes for Cisplatin (CNA): (64, 758)



In [39]:
# GDSC response values

Y = GDSCRv2['response'].values
print(Y)

[0 0 0 ... 0 1 0]


In [40]:
# change directory to read TCGA response dataset (Cisplatin)

os.chdir(dataset_dir + "response")
os.getcwd()

'/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/response'

In [41]:
# read TCGA response dataset (Cisplatin)

TCGARcisplatin = pd.read_csv("TCGA_response.Cisplatin.tsv", 
                      sep = "\t", 
                      index_col=0, 
                      decimal = ",")
TCGARcisplatin.dropna(inplace=True)
TCGARcisplatin.rename(mapper=str, axis='index', inplace=True)
d = {"R": 0, "S": 1}
TCGARcisplatin["response"] = TCGARcisplatin.loc[:, "response"].apply(lambda x: d[x])
Ytscisplatin = TCGARcisplatin["response"].values    

print(f"There are {Ytscisplatin.shape[0]} samples with response data in the TCGA (Cisplatin)\n")
Ytscisplatin

There are 64 samples with response data in the TCGA (Cisplatin)



array([1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [42]:
# TCGA cohort table with TCGA response dataset (Cisplatin)

dat_tcga = TCGARcisplatin["cohort"]

dat_tcga.value_counts()

cohort
HNSC    37
CESC    20
STAD     3
SARC     2
UCS      2
Name: count, dtype: int64

In [43]:
# filter HNSC cohort samples (Cisplatin)

filter = (TCGARcisplatin["cohort"] == "HNSC")
tcga_hnsc_samples = dat_tcga.loc[filter,].index
tcga_hnsc_samples

Index(['TCGA-BA-A4IF-01A', 'TCGA-BA-A4IH-01A', 'TCGA-BA-A6D8-01A',
       'TCGA-BA-A6DA-01A', 'TCGA-BA-A6DD-01A', 'TCGA-BA-A6DI-01A',
       'TCGA-BA-A6DJ-01A', 'TCGA-CN-6998-01A', 'TCGA-CN-A63T-01A',
       'TCGA-CN-A641-01A', 'TCGA-CN-A6V3-01A', 'TCGA-CN-A6V6-01A',
       'TCGA-CN-A6V7-01A', 'TCGA-CQ-A4CH-01A', 'TCGA-D6-A6EK-01A',
       'TCGA-F7-A50J-01A', 'TCGA-HD-8314-01A', 'TCGA-IQ-A61E-01A',
       'TCGA-IQ-A61G-01A', 'TCGA-IQ-A61J-01A', 'TCGA-IQ-A61O-01A',
       'TCGA-KU-A6H7-01A', 'TCGA-MZ-A5BI-01A', 'TCGA-MZ-A6I9-01A',
       'TCGA-MZ-A7D7-01A', 'TCGA-P3-A6SW-01A', 'TCGA-P3-A6T2-01A',
       'TCGA-P3-A6T3-01A', 'TCGA-P3-A6T7-01A', 'TCGA-P3-A6T8-01A',
       'TCGA-T2-A6X0-01A', 'TCGA-TN-A7HJ-01A', 'TCGA-TN-A7HL-01A',
       'TCGA-UF-A7JC-01A', 'TCGA-UF-A7JF-01A', 'TCGA-UF-A7JJ-01A',
       'TCGA-UF-A7JK-01A'],
      dtype='object', name='sample')

In [44]:
# filter responses of HNSC cohort samples (Cisplatin)

Ytscisplatin_hnsc = TCGARcisplatin.loc[tcga_hnsc_samples,"response"].values
Ytscisplatin_hnsc

array([1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [45]:
# count of sensitive and resistance HNSC samples (Cisplatin)

TCGARcisplatin.loc[tcga_hnsc_samples,"response"].value_counts()

response
1    34
0     3
Name: count, dtype: int64

In [46]:
# define maximum iteration

max_iter = 50

In [47]:
# assign number of neurons and dropout rates found for layers from the saved models

hdm1 = 512
hdm2 = 256
hdm3 = 256
rate1 = 0.5
rate2 = 0.4
rate3 = 0.7
rate4 = 0.5

In [48]:
# load pre-trained models and make predictions

scalerGDSC = sk.StandardScaler()
scalerGDSC.fit(GDSCEv2.values)
X_trainE = scalerGDSC.transform(GDSCEv2.values)

X_testEcisplatin = scalerGDSC.transform(TCGAEcisplatin.values)    
X_testEcisplatin_hnsc = scalerGDSC.transform(TCGAEcisplatin.loc[tcga_hnsc_samples,].values)    

X_trainM = np.nan_to_num(GDSCMv2.values)
X_testMcisplatin = np.nan_to_num(TCGAMcisplatin.values.astype(np.float32))
X_testMcisplatin_hnsc = np.nan_to_num(TCGAMcisplatin.loc[tcga_hnsc_samples,].values.astype(np.float32))

X_trainC = np.nan_to_num(GDSCCv2.values)
X_testCcisplatin = np.nan_to_num(TCGACcisplatin.values)
X_testCcisplatin_hnsc = np.nan_to_num(TCGACcisplatin.loc[tcga_hnsc_samples,].values)

TX_testEcisplatin = torch.FloatTensor(X_testEcisplatin)
TX_testEcisplatin_hnsc = torch.FloatTensor(X_testEcisplatin_hnsc)

TX_testMcisplatin = torch.FloatTensor(X_testMcisplatin.astype(np.float32))
TX_testMcisplatin_hnsc = torch.FloatTensor(X_testMcisplatin_hnsc.astype(np.float32))

TX_testCcisplatin = torch.FloatTensor(X_testCcisplatin.astype(np.float32))
TX_testCcisplatin_hnsc = torch.FloatTensor(X_testCcisplatin_hnsc.astype(np.float32))

ty_testEcisplatin = torch.FloatTensor(Ytscisplatin.astype(int))
ty_testEcisplatin_hnsc = torch.FloatTensor(Ytscisplatin_hnsc.astype(int))

n_sampE, IE_dim = X_trainE.shape
n_sampM, IM_dim = X_trainM.shape
n_sampC, IC_dim = X_trainC.shape

h_dim1 = hdm1
h_dim2 = hdm2
h_dim3 = hdm3        
Z_in = h_dim1 + h_dim2 + h_dim3

class AEE(nn.Module):
    def __init__(self):
        super(AEE, self).__init__()
        self.EnE = torch.nn.Sequential(
            nn.Linear(IE_dim, h_dim1),
            nn.BatchNorm1d(h_dim1),
            nn.ReLU(),
            nn.Dropout(rate1))
    def forward(self, x):
        output = self.EnE(x)
        return output  


class AEM(nn.Module):
    def __init__(self):
        super(AEM, self).__init__()
        self.EnM = torch.nn.Sequential(
            nn.Linear(IM_dim, h_dim2),
            nn.BatchNorm1d(h_dim2),
            nn.ReLU(),
            nn.Dropout(rate2))
    def forward(self, x):
        output = self.EnM(x)
        return output       

class AEC(nn.Module):
    def __init__(self):
        super(AEC, self).__init__()
        self.EnC = torch.nn.Sequential(
            nn.Linear(IC_dim, h_dim3),
            nn.BatchNorm1d(h_dim3),
            nn.ReLU(),
            nn.Dropout(rate3))
    def forward(self, x):
        output = self.EnC(x)
        return output 
        
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.FC = torch.nn.Sequential(
            nn.Linear(Z_in, 1),
            nn.Dropout(rate4),
            nn.Sigmoid())
    def forward(self, x):
        return self.FC(x)

torch.cuda.manual_seed_all(42)

AutoencoderE = torch.load(models_dir + "Exprs_Cisplatin_GDSC_TCGA_DNA_REPi_Expression_Mutation_CNA.pt")
AutoencoderM = torch.load(models_dir + "Mut_Cisplatin_GDSC_TCGA_DNA_REPi_Expression_Mutation_CNA.pt")
AutoencoderC = torch.load(models_dir + "CNA_Cisplatin_GDSC_TCGA_DNA_REPi_Expression_Mutation_CNA.pt")

Clas = torch.load(models_dir + "Class_Cisplatin_GDSC_TCGA_DNA_REPi_Expression_Mutation_CNA.pt")

AutoencoderE.eval()
AutoencoderM.eval()
AutoencoderC.eval()
Clas.eval()

ZEX = AutoencoderE(torch.FloatTensor(X_trainE))
ZMX = AutoencoderM(torch.FloatTensor(X_trainM))
ZCX = AutoencoderC(torch.FloatTensor(X_trainC))
ZTX = torch.cat((ZEX, ZMX, ZCX), 1)
ZTX = F.normalize(ZTX, p=2, dim=0)
PredX = Clas(ZTX)
AUCT = roc_auc_score(Y, PredX.detach().numpy())
#print(f"GDSC - Training AUC (TCGA_DNA_REPi): {round(AUCT,2)}")
AUCTaucpr = average_precision_score(Y, PredX.detach().numpy())
#print(f"GDSC - Training AUCPR (TCGA_DNA_REPi): {round(AUCTaucpr,2)}\n")

ZETcisplatin = AutoencoderE(TX_testEcisplatin)
ZMTcisplatin = AutoencoderM(TX_testMcisplatin)
ZCTcisplatin = AutoencoderC(TX_testCcisplatin)
ZTTcisplatin = torch.cat((ZETcisplatin, ZMTcisplatin, ZCTcisplatin), 1)
ZTTcisplatin = F.normalize(ZTTcisplatin, p=2, dim=0)
PredTcisplatin = Clas(ZTTcisplatin)
AUCTcisplatin = roc_auc_score(Ytscisplatin.astype(int), PredTcisplatin.detach().numpy())
#print(f"TCGA - Test AUC (Cisplatin): {round(AUCTcisplatin,2)}")
AUCTcisplatin_aucpr = average_precision_score(Ytscisplatin.astype(int), PredTcisplatin.detach().numpy())
#print(f"TCGA - Test AUCPR (Cisplatin): {round(AUCTcisplatin_aucpr,2)}\n")

ZETcisplatin_hnsc = AutoencoderE(TX_testEcisplatin_hnsc)
ZMTcisplatin_hnsc = AutoencoderM(TX_testMcisplatin_hnsc)
ZCTcisplatin_hnsc = AutoencoderC(TX_testCcisplatin_hnsc)
ZTTcisplatin_hnsc = torch.cat((ZETcisplatin_hnsc, ZMTcisplatin_hnsc, ZCTcisplatin_hnsc), 1)
ZTTcisplatin_hnsc = F.normalize(ZTTcisplatin_hnsc, p=2, dim=0)
PredTcisplatin_hnsc = Clas(ZTTcisplatin_hnsc)
AUCTcisplatin_hnsc = roc_auc_score(Ytscisplatin_hnsc.astype(int), PredTcisplatin_hnsc.detach().numpy())
#print(f"TCGA - Test AUC (Cisplatin, HNSC): {round(AUCTcisplatin_hnsc,2)}")
AUCTcisplatin_hnsc_aucpr = average_precision_score(Ytscisplatin_hnsc.astype(int), PredTcisplatin_hnsc.detach().numpy())
#print(f"TCGA - Test AUCPR (Cisplatin, HNSC): {round(AUCTcisplatin_hnsc_aucpr,2)}\n")

AUCT_finetuned = dict()
AUCTaucpr_finetuned = dict()
AUCTcisplatin_finetuned = dict()
AUCTcisplatin_hnsc_finetuned = dict()
AUCTcisplatin_aucpr_finetuned = dict()
AUCTcisplatin_hnsc_aucpr_finetuned = dict()
for iter in range(max_iter):
   iter += 1  

   AutoencoderE_finetuned = torch.load(models_dir + f"Finetuned_Models/Exprs_Cisplatin_GDSC_TCGA_DNA_REPi_Expression_Mutation_CNA_Finetuned_{iter}.pt")
   AutoencoderM_finetuned = torch.load(models_dir + f"Finetuned_Models/Mut_Cisplatin_GDSC_TCGA_DNA_REPi_Expression_Mutation_CNA_Finetuned_{iter}.pt")
   AutoencoderC_finetuned = torch.load(models_dir + f"Finetuned_Models/CNA_Cisplatin_GDSC_TCGA_DNA_REPi_Expression_Mutation_CNA_Finetuned_{iter}.pt")
   Clas_finetuned = torch.load(models_dir + f"Finetuned_Models/Class_Cisplatin_GDSC_TCGA_DNA_REPi_Expression_Mutation_CNA_Finetuned_{iter}.pt")
   
   AutoencoderE_finetuned.eval()
   AutoencoderM_finetuned.eval()
   AutoencoderC_finetuned.eval()
   Clas_finetuned.eval()
      
   ZEX = AutoencoderE_finetuned(torch.FloatTensor(X_trainE))
   ZMX = AutoencoderM_finetuned(torch.FloatTensor(X_trainM))
   ZCX = AutoencoderC_finetuned(torch.FloatTensor(X_trainC))
   ZTX = torch.cat((ZEX, ZMX, ZCX), 1)
   ZTX = F.normalize(ZTX, p=2, dim=0)
   PredX = Clas_finetuned(ZTX)
   AUCT_finetuned[iter] = roc_auc_score(Y, PredX.detach().numpy())
   AUCTaucpr_finetuned[iter] = average_precision_score(Y, PredX.detach().numpy())
   
   ZETcisplatin = AutoencoderE_finetuned(TX_testEcisplatin)
   ZMTcisplatin = AutoencoderM_finetuned(TX_testMcisplatin)
   ZCTcisplatin = AutoencoderC_finetuned(TX_testCcisplatin)
   ZTTcisplatin = torch.cat((ZETcisplatin, ZMTcisplatin, ZCTcisplatin), 1)
   ZTTcisplatin = F.normalize(ZTTcisplatin, p=2, dim=0)
   PredTcisplatin = Clas_finetuned(ZTTcisplatin)
   AUCTcisplatin_finetuned[iter] = roc_auc_score(Ytscisplatin.astype(int), PredTcisplatin.detach().numpy())
   AUCTcisplatin_aucpr_finetuned[iter] = average_precision_score(Ytscisplatin.astype(int), PredTcisplatin.detach().numpy())

   ZETcisplatin_hnsc = AutoencoderE_finetuned(TX_testEcisplatin_hnsc)
   ZMTcisplatin_hnsc = AutoencoderM_finetuned(TX_testMcisplatin_hnsc)
   ZCTcisplatin_hnsc = AutoencoderC_finetuned(TX_testCcisplatin_hnsc)
   ZTTcisplatin_hnsc = torch.cat((ZETcisplatin_hnsc, ZMTcisplatin_hnsc, ZCTcisplatin_hnsc), 1)
   ZTTcisplatin_hnsc = F.normalize(ZTTcisplatin_hnsc, p=2, dim=0)
   PredTcisplatin_hnsc = Clas_finetuned(ZTTcisplatin_hnsc)
   AUCTcisplatin_hnsc_finetuned[iter] = roc_auc_score(Ytscisplatin_hnsc.astype(int), PredTcisplatin_hnsc.detach().numpy())
   AUCTcisplatin_hnsc_aucpr_finetuned[iter] = average_precision_score(Ytscisplatin_hnsc.astype(int), PredTcisplatin_hnsc.detach().numpy())

max_key = max(AUCTcisplatin_aucpr_finetuned, key=AUCTcisplatin_aucpr_finetuned.get)
max_key_hnsc = max(AUCTcisplatin_hnsc_aucpr_finetuned, key=AUCTcisplatin_hnsc_aucpr_finetuned.get)

print(f"GDSC - Training AUC (DNA_REPi): {round(AUCT,2)}")
print(f"GDSC - Training AUCPR (DNA_REPi): {round(AUCTaucpr,2)}")
print(f"TCGA - Test AUC (Cisplatin): {round(AUCTcisplatin,2)}")
print(f"TCGA - Test AUCPR (Cisplatin): {round(AUCTcisplatin_aucpr,2)}")
print(f"TCGA - Test AUC (Cisplatin, HNSC): {round(AUCTcisplatin_hnsc,2)}")
print(f"TCGA - Test AUCPR (Cisplatin, HNSC): {round(AUCTcisplatin_hnsc_aucpr,2)}\n")

print(f"GDSC - Training AUC (DNA_REPi, Finetuned): {round(AUCT_finetuned[max_key],2)}")
print(f"GDSC - Training AUCPR (DNA_REPi, Finetuned): {round(AUCTaucpr_finetuned[max_key],2)}")
print(f"TCGA - Test AUC (Cisplatin, Finetuned): {round(AUCTcisplatin_finetuned[max_key],2)}")
print(f"TCGA - Test AUCPR (Cisplatin, Finetuned): {round(AUCTcisplatin_aucpr_finetuned[max_key],2)}\n")

print(f"GDSC - Training AUC (DNA_REPi, Finetuned): {round(AUCT_finetuned[max_key_hnsc],2)}")
print(f"GDSC - Training AUCPR (DNA_REPi, Finetuned): {round(AUCTaucpr_finetuned[max_key_hnsc],2)}")
print(f"TCGA - Test AUC (Cisplatin, NSCLC, Finetuned): {round(AUCTcisplatin_hnsc_finetuned[max_key_hnsc],2)}")
print(f"TCGA - Test AUCPR (Cisplatin, NSCLC, Finetuned): {round(AUCTcisplatin_hnsc_aucpr_finetuned[max_key_hnsc],2)}\n\n")

print("Models for maximum finetuning (Cisplatin)")
print(f"Exprs_Cisplatin_GDSC_TCGA_DNA_REPi_Expression_Mutation_CNA_Finetuned_{max_key}.pt")
print(f"Mut_Cisplatin_GDSC_TCGA_DNA_REPi_Expression_Mutation_CNA_Finetuned_{max_key}.pt")
print(f"CNA_Cisplatin_GDSC_TCGA_DNA_REPi_Expression_Mutation_CNA_Finetuned_{max_key}.pt")
print(f"Class_Cisplatin_GDSC_TCGA_DNA_REPi_Expression_Mutation_CNA_Finetuned_{max_key}.pt")


print("\nModels for maximum finetuning (Cisplatin, HNSC)")
print(f"Exprs_Cisplatin_GDSC_TCGA_DNA_REPi_Expression_Mutation_CNA_Finetuned_{max_key_hnsc}.pt")
print(f"Mut_Cisplatin_GDSC_TCGA_DNA_REPi_Expression_Mutation_CNA_Finetuned_{max_key_hnsc}.pt")
print(f"CNA_Cisplatin_GDSC_TCGA_DNA_REPi_Expression_Mutation_CNA_Finetuned_{max_key_hnsc}.pt")
print(f"Class_Cisplatin_GDSC_TCGA_DNA_REPi_Expression_Mutation_CNA_Finetuned_{max_key_hnsc}.pt")

GDSC - Training AUC (DNA_REPi): 0.84
GDSC - Training AUCPR (DNA_REPi): 0.36
TCGA - Test AUC (Cisplatin): 0.73
TCGA - Test AUCPR (Cisplatin): 0.96
TCGA - Test AUC (Cisplatin, HNSC): 0.58
TCGA - Test AUCPR (Cisplatin, HNSC): 0.93

GDSC - Training AUC (DNA_REPi, Finetuned): 0.84
GDSC - Training AUCPR (DNA_REPi, Finetuned): 0.35
TCGA - Test AUC (Cisplatin, Finetuned): 0.76
TCGA - Test AUCPR (Cisplatin, Finetuned): 0.97

GDSC - Training AUC (DNA_REPi, Finetuned): 0.84
GDSC - Training AUCPR (DNA_REPi, Finetuned): 0.36
TCGA - Test AUC (Cisplatin, NSCLC, Finetuned): 0.66
TCGA - Test AUCPR (Cisplatin, NSCLC, Finetuned): 0.95


Models for maximum finetuning (Cisplatin)
Exprs_Cisplatin_GDSC_TCGA_DNA_REPi_Expression_Mutation_CNA_Finetuned_1.pt
Mut_Cisplatin_GDSC_TCGA_DNA_REPi_Expression_Mutation_CNA_Finetuned_1.pt
CNA_Cisplatin_GDSC_TCGA_DNA_REPi_Expression_Mutation_CNA_Finetuned_1.pt
Class_Cisplatin_GDSC_TCGA_DNA_REPi_Expression_Mutation_CNA_Finetuned_1.pt

Models for maximum finetuning (Cisplati

In [49]:
# show expression layer parameters

AutoencoderE

AEE(
  (EnE): Sequential(
    (0): Linear(in_features=758, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
  )
)

In [50]:
# show mutation layer parameters

AutoencoderM

AEM(
  (EnM): Sequential(
    (0): Linear(in_features=758, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
  )
)

In [51]:
# show CNA layer parameters

AutoencoderC

AEC(
  (EnC): Sequential(
    (0): Linear(in_features=758, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.7, inplace=False)
  )
)

In [52]:
# show classification layer parameters

Clas

Classifier(
  (FC): Sequential(
    (0): Linear(in_features=1024, out_features=1, bias=True)
    (1): Dropout(p=0.5, inplace=False)
    (2): Sigmoid()
  )
)

In [53]:
# create a dataframe with the prediction results 

emc = {"Data": ["Expr + Mut + CNA", "Expr + Mut + CNA (Fine-tuned)"],
       "AUC (GDSC)": [round(AUCT, 2), round(AUCT_finetuned[max_key], 2)],
       "AUCPR (GDSC)": [round(AUCTaucpr,2), round(AUCTaucpr_finetuned[max_key],2)],
       "AUC (TCGA)": [round(AUCTcisplatin,2), round(AUCTcisplatin_finetuned[max_key],2)],
       "AUCPR (TCGA)":  [round(AUCTcisplatin_aucpr,2), round(AUCTcisplatin_aucpr_finetuned[max_key],2)],
       "AUC (TCGA-HNSC)":  [round(AUCTcisplatin_hnsc,2), round(AUCTcisplatin_hnsc_finetuned[max_key_hnsc],2)],
       "AUCPR (TCGA-HNSC)": [round(AUCTcisplatin_hnsc_aucpr,2), round(AUCTcisplatin_hnsc_aucpr_finetuned[max_key_hnsc],2)],
       "Sample Size (GDSC)": [GDSCEv2.shape[0], GDSCEv2.shape[0]],
       "Feature Size (GDSC)": [GDSCEv2.shape[1], GDSCEv2.shape[1]] 
}
emc_dataframe = pd.DataFrame.from_dict(emc, orient='index').transpose()  
emc_dataframe.to_csv(save_results_to + "GDSC_TCGA_Expression_Mutation_CNA_DNA_REPi_Cisplatin.tsv",
                    sep = "\t",
                    index = False)

emc_dataframe

,Data,AUC (GDSC),AUCPR (GDSC),AUC (TCGA),AUCPR (TCGA),AUC (TCGA-HNSC),AUCPR (TCGA-HNSC),Sample Size (GDSC),Feature Size (GDSC)
0,Expr + Mut + CNA,0.84,0.36,0.73,0.96,0.58,0.93,9435,758
1,Expr + Mut + CNA (Fine-tuned),0.84,0.35,0.76,0.97,0.66,0.95,9435,758


In [54]:
# change directory

os.chdir(save_results_to)
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/Transfer Learning/Drugs with same pathways/TCGA_DNA_REPi/Expression_Mutation_CNA/Predictions


In [55]:
# save predictions

file = open('GDSC_TCGA_DEGs - TCGA_DNA_REPi_Cisplatin Predictions (Expression, Mutation, and CNA).txt', 'w')
file.write(f"GDSC Training (EMC) AUC (TCGA_DNA_REPi): {round(AUCT,2)}\n")
file.write(f"GDSC Training (EMC) AUCPR (TCGA_DNA_REPi): {round(AUCTaucpr,2)}\n\n")

file.write(f"TCGA Test (EMC) AUC (Cisplatin): {round(AUCTcisplatin,2)}\n")
file.write(f"TCGA Test (EMC) AUCPR (Cisplatin): {round(AUCTcisplatin_aucpr,2)}\n\n")

file.close()